In [ ]:
import sys
import pandas as pd
import numpy as np
sys.path.append('../src/')

In [ ]:
APPLIANCE = ['fridge', 'dish_washer', 'washing_machine']
THRESHOLD = [50., 10., 20.]
MIN_ON = [1., 30., 30.]
MIN_OFF = [1., 30., 3.]

METER = 'aggregate'
SEQ_LEN = 60*8
BORDER = 16
BATCH_SIZE = 32

MAX_POWER = 2000.

In [ ]:
from utils.preprocessing import get_status
houses=[1,2,5]
original_meter = []
ds_appliance = []
ds_status = []
for i in houses:
    ds = pd.read_feather('../data/ukdale/feather_files/UKDALE_%d_train.feather' %(i))
    ds.set_index('datetime', inplace=True)
  
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    original_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(original_meter[i]) for i in range(3)]

#### Filtering

In [ ]:
from utils.preprocessing import chain2filter
THRESHOLDS=[50,100,150,200,250,300]
datafilter=[]
for threshold in THRESHOLDS:
    datafilter.append(chain2filter(original_meter, threshold))

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from utils.preprocessing import Power

ds_house_total  = [Power(original_meter[i], ds_appliance[i], ds_status[i], 
                         SEQ_LEN, BORDER, MAX_POWER, False) for i in range(3)]
dl_test = DataLoader(dataset = ds_house_total[1], batch_size = 32, shuffle=False)

dl_house_test = [DataLoader(dataset = ds_house_total[1], batch_size = 32, shuffle=False)]
dl_house_total = [DataLoader(dataset = ds_house_total[i], batch_size = 32, shuffle=False) for i in range(3)]

for i in range(6):
    globals()["ds_house_filter_" + str(THRESHOLDS[i])] = [Power(datafilter[i][j], ds_appliance[j], ds_status[j], 
                        SEQ_LEN, BORDER, MAX_POWER, False) for j in range(3)]
    globals()["dl_house_filter_" + str(THRESHOLDS[i])] = [DataLoader(dataset = globals()["ds_house_filter_" + str(THRESHOLDS[i])][j], batch_size = 32, shuffle=False) for j in range(3)]

In [1]:
dataiter = iter(dl_house_total[1])
dataiter_filtered_100 = iter(dl_house_filter_100[1])

NameError: name 'dl_house_total' is not defined

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(15,8))
#x, y, s = dataiter.next()
a = 0
for i in range(100):
    x, y, s = dataiter.next()
    if y[0,:,a].sum() > 0:
        break
    if s[0,:,a].sum() > 0:
        break
plt.plot(np.arange(-BORDER, SEQ_LEN + BORDER), x[0,:].detach().numpy(), 'k-')
plt.plot(y[0,:,a].detach().numpy())
plt.plot(s[0,:,a].detach().numpy())
plt.ylim([-0.5,1.5])

In [ ]:
plt.figure(figsize=(15,8))
#x, y, s = dataiter.next()
a = 2
for i in range(100):
    x, y, s = dataiter_filtered_100.next()
    if y[0,:,a].sum() > 0:
        break
    if s[0,:,a].sum() > 0:
        break
plt.plot(np.arange(-BORDER, SEQ_LEN + BORDER), x[0,:].detach().numpy(), 'k-')
plt.plot(y[0,:,a].detach().numpy())
plt.plot(s[0,:,a].detach().numpy())
plt.ylim([-0.5,1.5])

In [ ]:
pm = (ds_appliance[0].sum() / ds_status[0].sum()).values

plt.figure(figsize=(12,8))

i = 24*60*600 +60*4

x = (ds_meter[0][i-BORDER:i+24*60+BORDER].values / MAX_POWER).copy()
x -=x.mean()

p = torch.sigmoid(model(torch.from_numpy(x).view(1,1,-1))).permute(0,2,1).cpu().detach().numpy()[0]
ds_probs = pd.DataFrame(p, index=ds_meter[0][i:i+24*60].index)

s = np.zeros_like(p)
for a in range(3):
    s[:,a] = get_status(p[:,a], 0.5, MIN_OFF[a], MIN_ON[a])

ds_activation = pd.DataFrame(s, index=ds_meter[0][i:i+24*60].index)
ds_y = ds_activation * pm

#plt.subplots(nrows=2, ncols=1, sharex=True)

plt.subplot(2,1,1)
plt.plot(ds_meter[0][i:i+24*60], color='k', label='Aggregate')
plt.plot(ds_y.index,ds_y[0], label='Fridge')
plt.plot(ds_y.index,ds_y[1], label='Dishwasher')
plt.plot(ds_y.index,ds_y[2], label='Washing machine')
plt.ylabel('Power [W]')
plt.legend(loc=2)
plt.ylim((0,3500))
#plt.xticks([])

plt.subplot(2,1,2)
plt.plot(ds_appliance[0][i:i+24*60])
plt.ylabel('Power [W]')
plt.ylim((0,3500))
plt.legend(('Fridge', 'Dishwasher', 'Washing machine'), loc=2)

In [ ]:
pm = (ds_appliance[1].sum() / ds_status[1].sum()).values

plt.figure(figsize=(12,8))

i = 24*60*123 +60*4

x = (ds_meter[1][i-BORDER:i+24*60+BORDER].values / MAX_POWER).copy()
x -=x.mean()

p = torch.sigmoid(model(torch.from_numpy(x).view(1,1,-1))).permute(0,2,1).cpu().detach().numpy()[0]
ds_probs = pd.DataFrame(p, index=ds_meter[1][i:i+24*60].index)

s = np.zeros_like(p)
for a in range(3):
    s[:,a] = get_status(p[:,a], 0.5, MIN_OFF[a], MIN_ON[a])

ds_activation = pd.DataFrame(s, index=ds_meter[1][i:i+24*60].index)
ds_y = ds_activation * pm

plt.subplot(2,1,1)
plt.plot(ds_meter[1][i:i+24*60], color='k', label='Aggregate')
plt.plot(ds_y.index,ds_y[0], label='Fridge')
plt.plot(ds_y.index,ds_y[1], label='Dishwasher')
plt.plot(ds_y.index,ds_y[2], label='Washing machine')
plt.ylabel('Power [W]')
plt.legend(loc=2)
plt.ylim((0,3500))
#plt.xticks([])

plt.subplot(2,1,2)
plt.plot(ds_appliance[1][i:i+24*60])
plt.ylabel('Power [W]')
plt.ylim((0,3500))
plt.legend(('Fridge', 'Dishwasher', 'Washing machine'), loc=2)